In [ ]:
from google.colab import drive 

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [ ]:
filename_feature='/content/gdrive/MyDrive/aicomm/titanic_feature.csv' 
filename_survival='/content/gdrive/MyDrive/aicomm/titanic_survival.csv' 

features = pd.read_csv(filename_feature, index_col = 0)
survival = pd.read_csv(filename_survival, index_col = 0)

In [ ]:
train_features, test_features, train_labels, test_labels = train_test_split(features.values, survival.values, train_size = 0.8)

train_labels = train_labels.reshape(-1)
test_labels = test_labels.reshape(-1)


scaler = StandardScaler()

train_features = scaler.fit_transform(train_features)
test_features = scaler.transform(test_features)

test_length = test_features.shape[0]

In [ ]:
class DataSet(Dataset):
  def __init__(self, train_x, train_y):
    self.X = train_x
    self.y = train_y
  def __getitem__(self, item):
    x_t = self.X[item]
    y_t = self.y[item]
    return x_t, y_t
  def __len__(self):
    return len(self.X)

In [ ]:
dataset_train = DataSet(train_features, train_labels)
dataset_test = DataSet(test_features, test_labels)

In [ ]:
batch_size = 1
train_loader = DataLoader(dataset = dataset_train, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = dataset_test, batch_size = batch_size, shuffle = False)

In [ ]:
class Net(nn.Module):
  def __init__(self):
    super(Net, self).__init__()
    self.fc1 = nn.Linear(in_features=4, out_features=4)
    self.sigmoid = nn.Sigmoid()
    self.fc2 = nn.Linear(in_features = 4, out_features = 4)
    self.fc3 = nn.Linear(in_features = 4, out_features = 2)
  def forward(self, x):
    x = self.fc1(x)
    x = self.sigmoid(x)
    x = self.fc2(x)
    x = self.sigmoid(x)
    x = self.fc3(x)
    return x.squeeze(1)

In [ ]:
net = Net()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr = 1e-3) 
training_loss = []

In [ ]:
def training_process(net, data_loader):
  net.train()
  train_loss = 0
  loss = 0
  for batch_idx, (data, label) in enumerate(data_loader):
    data, label = Variable(data.float()).to(device), Variable(label.type(torch.LongTensor)).to(device)
    optimizer.zero_grad()
    
    output = net.forward(data)
    loss = criterion(output, label)
    loss.backward()
    train_loss += loss.item()
    optimizer.step()

  return net, train_loss

In [ ]:
def validation_process(net, data_loader):
  net.eval()
  with torch.no_grad():
    accuracy_total = 0
    for batch_idx, (data, label) in enumerate(data_loader):
      data, label = Variable(data.float()).to(device), Variable(label.type(torch.LongTensor)).to(device)
      output = net.forward(data)

      if torch.argmax(output, 1) == label:
        correct_prediction = 1
      else:
        correct_prediction = 0

      accuracy_total += correct_prediction

  return accuracy_total

In [ ]:
training_loss = []
epochs = 50
best_accuracy = 0
for epoch in range(epochs):
  net, train_loss = training_process(net, train_loader)

  print('Epoch : %d // Loss : %.6f' %(epoch, train_loss))
  training_loss.append(train_loss)

  accuracy = validation_process(net, test_loader)
  print('Accuracy:%.2f' %(100*accuracy/test_length))
print("학습 끝!")

Epoch : 0 // Loss : 386.113181
Accuracy:60.84
Epoch : 1 // Loss : 380.687966
Accuracy:60.84
Epoch : 2 // Loss : 371.714733
Accuracy:60.84
Epoch : 3 // Loss : 357.230048
Accuracy:65.73
Epoch : 4 // Loss : 337.540441
Accuracy:74.83
Epoch : 5 // Loss : 316.452564
Accuracy:74.83
Epoch : 6 // Loss : 299.193523
Accuracy:74.83
Epoch : 7 // Loss : 286.891729
Accuracy:74.83
Epoch : 8 // Loss : 278.394322
Accuracy:74.83
Epoch : 9 // Loss : 272.984337
Accuracy:74.13
Epoch : 10 // Loss : 269.406835
Accuracy:74.13
Epoch : 11 // Loss : 266.915445
Accuracy:74.13
Epoch : 12 // Loss : 265.073746
Accuracy:73.43
Epoch : 13 // Loss : 263.621212
Accuracy:74.13
Epoch : 14 // Loss : 262.469111
Accuracy:74.13
Epoch : 15 // Loss : 261.232812
Accuracy:74.83
Epoch : 16 // Loss : 260.211599
Accuracy:77.62
Epoch : 17 // Loss : 259.250940
Accuracy:77.62
Epoch : 18 // Loss : 258.744844
Accuracy:77.62
Epoch : 19 // Loss : 257.483356
Accuracy:76.22
Epoch : 20 // Loss : 256.493450
Accuracy:78.32
Epoch : 21 // Loss : 25